In [ ]:
!pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 51.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.tree import DecisionTreeRegressor
from pulp import LpMinimize, LpProblem, LpVariable, lpSum

# Function to read data from Excel
def read_data(file_path):
    df = pd.read_excel(file_path)
    return df

# Function to preprocess the data
def preprocess_data(df):
    # Encoding categorical variables
    encoder = OneHotEncoder(sparse=False)
    encoded_features = encoder.fit_transform(df[['courier', 'WH', 'order_destination']])
    encoded_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(['courier', 'WH', 'order_destination']))

    # Normalizing numerical features
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(df[['TAT', 'charges']])
    scaled_df = pd.DataFrame(scaled_features, columns=['TAT', 'charges'])

    # Combining encoded and scaled features
    final_df = pd.concat([encoded_df, scaled_df], axis=1)

    return final_df, encoder, scaler

# Read the dataset
file_path = 'sample_orders_dataset.xlsx'
df = read_data(file_path)
final_df, encoder, scaler = preprocess_data(df)

# Split features and target variables
X = final_df.drop(columns=['TAT', 'charges'])
y_tat = final_df['TAT']
y_cost = final_df['charges']

# Train ML models
tat_model = DecisionTreeRegressor()
tat_model.fit(X, y_tat)

cost_model = DecisionTreeRegressor()
cost_model.fit(X, y_cost)

print("Models trained successfully.")


Models trained successfully.


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
def create_and_solve_optimization(order_id):
    order_row = df[df['order_id'] == order_id].iloc[0]
    selected_wh = order_row['WH']
    selected_dest = order_row['order_destination']

    filtered_df = df[(df['WH'] == selected_wh) & (df['order_destination'] == selected_dest)].reset_index(drop=True)

    # Encode and scale the filtered data
    encoded_features = encoder.transform(filtered_df[['courier', 'WH', 'order_destination']])
    scaled_features = scaler.transform(filtered_df[['TAT', 'charges']])

    filtered_final_df = pd.concat([pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(['courier', 'WH', 'order_destination'])),
                                   pd.DataFrame(scaled_features, columns=['TAT', 'charges'])], axis=1)

    # Drop target variables before prediction
    filtered_final_df_for_prediction = filtered_final_df.drop(columns=['TAT', 'charges'])

    orders = filtered_final_df.index.tolist()
    couriers = filtered_df['courier'].unique().tolist()

    # Predicted values
    cost_pred = cost_model.predict(filtered_final_df_for_prediction) # Use the DataFrame without target variables
    tat_pred = tat_model.predict(filtered_final_df_for_prediction) # Use the DataFrame without target variables

    results = []
    for i in orders:
        courier = filtered_df.iloc[i]['courier']
        courier_cost = cost_pred[i]
        courier_tat = tat_pred[i]
        results.append((courier, courier_cost, courier_tat))

    # Sorting results based on the objective function (weighted sum of cost and TAT)
    alpha = 0.5  # weight for cost
    beta = 0.5   # weight for TAT
    results.sort(key=lambda x: (alpha * x[1] + beta * x[2]))

    # Ensure unique couriers
    unique_results = {}
    for courier, cost, tat in results:
        if courier not in unique_results:
            unique_results[courier] = (cost, tat)

    sorted_unique_results = sorted(unique_results.items(), key=lambda x: (alpha * x[1][0] + beta * x[1][1]))

    return sorted_unique_results

In [ ]:
def select_best_carrier():
    order_id = int(input("Enter Order ID: "))

    if order_id not in df['order_id'].values:
        print("Invalid Order ID")
        return

    results = create_and_solve_optimization(order_id)

    if results:
        print(f"Ranked couriers for Order ID {order_id}:")
        for rank, (courier, (cost, tat)) in enumerate(results, start=1):
            print(f"Rank {rank}: Courier: {courier}, Cost: {cost:.2f}, TAT: {tat:.2f}")
    else:
        print(f"No suitable courier found for Order ID {order_id}")

select_best_carrier()

Enter Order ID: 67
Ranked couriers for Order ID 67:
Rank 1: Courier: CourierC, Cost: -0.14, TAT: 0.06
Rank 2: Courier: CourierB, Cost: 0.01, TAT: 0.02
Rank 3: Courier: CourierA, Cost: 0.04, TAT: -0.00
Rank 4: Courier: CourierD, Cost: 0.03, TAT: 0.04
